In [ ]:
import pandas as pd
import io
import zipfile
import requests
from xml.etree.ElementTree import parse
import pandas as pd
import time
import datetime

In [ ]:
!pip install cchardet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for cchardet: filename=cchardet-2.1.7-cp310-cp310-linux_x86_64.whl size=289401 sha256=e2821b046cb9c230f7b4e361d86f778f4445c30592ca43fc48c0183cc33d532d
  Stored in directory: /root/.cache/pip/wheels/ee/e0/ab/e01326f15c59438d080b1496dbab8091e952ec72f35e3c437e
Successfully built cchardet


In [ ]:
import requests
import zipfile
import os

url="https://opendart.fss.or.kr/api/corpCode.xml"
api_key="5016e83354f0447e5828f09ecb72a9c373a0bbf4"
params={'crtfc_key' : api_key}

response=requests.get(url, params=params).content

with open ('./id.zip','wb') as fp:
  fp.write(response.content)

zf=zipfile.ZipFile('./id.zip')
zf.extractall()
xml_path=os.path.abspath('./CORPCODE.xml')

In [ ]:
import cchardet
from bs4 import BeautifulSoup
with open(xml_path, 'r',encoding="utf-8") as fp:
  corpcode=BeautifulSoup(fp.read(),'lxml')

<ipython-input-5-f0e3594e9de3>:4: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  corpcode=BeautifulSoup(fp.read(),'lxml')


In [ ]:
import xml.etree.ElementTree as ET

tree=ET.parse(xml_path)
root=tree.getroot()
tags_list=root.findall('list')

In [ ]:
import pandas as pd

def convert(tag:ET.Element) -> dict:
  conv={}
  for child in list(tag):
    conv[child.tag]=child.text
  return conv

tags_list_dict = [convert(x) for x in tags_list]
df=pd.DataFrame(tags_list_dict)

In [ ]:
df #고유번호/정식명칭/종목코드/최종변경일자

,corp_code,corp_name,stock_code,modify_date
0,00434003,다코,,20170630
1,00430964,굿앤엘에스,,20170630
2,00388953,크레디피아제이십오차유동화전문회사,,20170630
3,00179984,연방건설산업,,20170630
4,00420143,브룩스피알아이오토메이션잉크,,20170630
...,...,...,...,...
107471,01835069,아로하,,20240510
107472,01839199,호프인터내셔널,,20240510
107473,01823392,포스텍네트웍스,,20240510
107474,01837003,농업회사법인복순도가,,20240510


In [ ]:
df[df['stock_code']!=' '] #종목코드가 있는 곳은 약 4천개뿐(==상장한 기업)

,corp_code,corp_name,stock_code,modify_date
1949,00260985,한빛네트,036720,20170630
1961,00264529,엔플렉스,040130,20170630
1962,00358545,동서정보기술,055000,20170630
2709,00231567,애드모바일,032600,20170630
3785,00359614,리더컴,056140,20170630
...,...,...,...,...
107407,00317104,라이온켐텍,171120,20240401
107408,00651901,에어부산,298690,20240401
107409,00661847,화인베스틸,133820,20240401
107410,01428203,케이씨씨글라스,344820,20240401


In [ ]:
#상장기업들만 !!
stock_df=df[df['stock_code']!=" "]
stock_df=stock_df[['corp_code','corp_name','stock_code']].drop_duplicates()
stock_df

,corp_code,corp_name,stock_code
1949,00260985,한빛네트,036720
1961,00264529,엔플렉스,040130
1962,00358545,동서정보기술,055000
2709,00231567,애드모바일,032600
3785,00359614,리더컴,056140
...,...,...,...
107407,00317104,라이온켐텍,171120
107408,00651901,에어부산,298690
107409,00661847,화인베스틸,133820
107410,01428203,케이씨씨글라스,344820
